## First of all we need to start testing models

In [5]:
import os
import zipfile
from email import policy
from email.parser import BytesParser
from pathlib import Path

To download only the mails pdf or atachment

In [32]:
def extract_eml_info(eml_path):
    # Open and parse the .eml file
    with open(eml_path, 'rb') as eml_file:
        msg = BytesParser(policy=policy.default).parse(eml_file)
    
    # Extract headers
    subject = msg.get('subject', '')
    from_ = msg.get('from', '')
    to = msg.get('to', '')
    date = msg.get('date', '')
    
    # Extract body (text and HTML)
    text_body = ""
    html_body = ""
    
    if msg.is_multipart():
        for part in msg.iter_parts():
            content_type = part.get_content_type()
            content_disposition = part.get('Content-Disposition')
            
            if content_type == 'text/plain' and not content_disposition:
                text_body = part.get_payload(decode=True).decode(part.get_content_charset())
            elif content_type == 'text/html' and not content_disposition:
                html_body = part.get_payload(decode=True).decode(part.get_content_charset())
            elif part.get_filename():  # Extract attachments
                filename = part.get_filename()
                with open(filename, 'wb') as f:
                    f.write(part.get_payload(decode=True))
    else:
        if msg.get_content_type() == 'text/plain':
            text_body = msg.get_payload(decode=True).decode(msg.get_content_charset())
        elif msg.get_content_type() == 'text/html':
            html_body = msg.get_payload(decode=True).decode(msg.get_content_charset())
    return [html_body]

In [8]:
data_path = Path("../data/")
mails_path = data_path / "mails"
zips_data = data_path / "zips"
mail_files = [mails_path / f for f in os.listdir(mails_path)]
zip_files = [zips_data / f for f in os.listdir(zips_data)]

In [34]:
mails_data = [extract_eml_info(i) for i in mail_files]

In [36]:
def extract_zip(zip_file_path, extract_to_folder):
    # Ensure the output directory exists
    os.makedirs(extract_to_folder, exist_ok=True)
    
    # Open the ZIP file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extract all the contents into the specified folder
        zip_ref.extractall(extract_to_folder)

In [44]:
for i in zip_files:
    extract_zip(i, data_path / "pdfs")

In [9]:
pdf_path = data_path / "pdfs"
pdf_files = [pdf_path / f for f in os.listdir(pdf_path) if f.endswith(".pdf")]

In [8]:
pdf_files

[PosixPath('../data/pdfs/fv09012646580082400000791.pdf'),
 PosixPath('../data/pdfs/fv0800242106021234008100243777.pdf'),
 PosixPath('../data/pdfs/ad08909411030122300010468.pdf'),
 PosixPath('../data/pdfs/fv090126465800824000007da.pdf'),
 PosixPath('../data/pdfs/fv090099897600824c0bf.pdf'),
 PosixPath('../data/pdfs/ad08909411030122300010673.pdf'),
 PosixPath('../data/pdfs/ad089094179401224000028AF.pdf'),
 PosixPath('../data/pdfs/ad089094179401224000028DF.pdf'),
 PosixPath('../data/pdfs/fv0900411441008230000dcb2.pdf'),
 PosixPath('../data/pdfs/ad0890941794012230001A094.pdf'),
 PosixPath('../data/pdfs/ad09011139020002400097930.pdf'),
 PosixPath('../data/pdfs/fv090126465800823000022a6.pdf'),
 PosixPath('../data/pdfs/ad0890941794012230001A138.pdf'),
 PosixPath('../data/pdfs/ad0860029002016240001e00b.pdf'),
 PosixPath('../data/pdfs/ad080022707101624000195cc.pdf'),
 PosixPath('../data/pdfs/ad0900378955016240000a5ef.pdf'),
 PosixPath('../data/pdfs/ad09011139020002400097958.pdf'),
 PosixPath('.

In [10]:
from langchain_community.document_loaders import UnstructuredPDFLoader

In [27]:
loader = UnstructuredPDFLoader(pdf_files[0])
data = loader.load()

In [2]:
import torch
from dotenv import load_dotenv
#from langchain_core.runnables import RunnablePassthrough
#from langchain.prompts import ChatPromptTemplate, PromptTemplate
#from langchain_core.output_parsers import StrOutputParser
#from langchain_community.chat_models import ChatOllama
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


2024-08-29 02:41:36.872094: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [66]:
#model_name = "mistral"
#model = ChatOllama(model="mistral")

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [58]:
load_dotenv()
token = os.getenv("HG_TOKEN")
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="gptq-8bit-32g-actorder_True") # which version of mode

/home/kaiki/anaconda3/envs/tensorflow/lib/python3.11/site-packages/transformers/modeling_utils.py:4674: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp

In [60]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [20]:
context_prompt = PromptTemplate(
    input_variables=["documento"],
    template="""Eres un asistente de facturas, solo hablas español, vas a recibir
    información de varias facturas y debes extraer la siguiente información y presentarla
    de esta forma \n
    
    Empresa: Nombre de la empresa del documento
    Nit: Nit de la empresa del documento
    Valor a pagar: Valor a pagar mostrado en el documento
    Numero de factura: Numero de factura encontrado en el documento
    Valor a pagar: Valor a pagar
    IVA: Valor de iva

    A partir del siguiente documento:
    {documento}
    """
)

In [21]:
chain = ( {"documento": RunnablePassthrough()} | context_prompt | model | StrOutputParser())

In [11]:
data[0].page_content

'FACTURA ELECTRÓNICA DE VENTA\n\nSODIMAC COLOMBIA S.A. NIT 800.242.106-2 Iva Régimen Común ICA Actividad Económica CIIU 4719 Grandes Contribuyentes Resolución 12220 26 DIC 2022 AUTORRETENEDORES Resolución DIAN 00931 de 29 ENE 2009 Agente de Retención de IVA\n\nN° 4008100243777\n\nINFORMACIÓN DEL CLIENTE\n\nRazón Social:\n\nNIT\n\nDirección\n\n: Armando Lopez Florian : 71.624.785 : CL 24 79 A 140\n\nFecha de Expedición Fecha de Vencimiento\n\nFecha de Validación\n\n: 2023/12/19 09:44:02\n\n: 2023/12/19\n\n: 2023/12/19\n\nTeléfono\n\n: 3136328200\n\nFecha de Compra\n\n: 2023/12/19\n\nCorreo\n\n: arlofl@gmail.com\n\nRemisión de Entrega\n\nDirección de Entrega\n\n:\n\nCiudad de Entrega\n\n: BOGOTÁ, D.C.\n\nForma de Pago\n\n: Contado\n\nNro O/C\n\nPlazo\n\n: 0 Dias de Pago\n\nTipo Moneda\n\n: COP\n\nNro. Interno\n\n: 33890618\n\nMedio de Pago\n\n. TARJ CRE/DEB\n\nCUFE\n\n: 4c2a88ad96c5c3f1fbaf3bf03385d9aa2f2c791a2ffc432e6825d0b9e2a895560bd91e478e1abeb215dc9de2768cf764\n\nObservaciones\n\n: 

In [30]:
chain.invoke(data[0].page_content)

TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not StringPromptValue

In [61]:
import re

In [120]:
template="""Eres un asistente de facturas, solo hablas español, vas a recibir
información de varias facturas y debes extraer la siguiente información y presentarla
de esta forma y solo con estos datos:

Empresa: Nombre de la empresa del documento
Nit: Nit de la empresa del documento
Valor a pagar: Valor a pagar mostrado en el documento
Numero de factura: Numero de factura encontrado en el documento
Valor a pagar: Valor a pagar
IVA: Valor de iva, este valor es el valor a pagar / 1.19
CUFE: CUFE del documento

A partir del siguiente documento:

"""
#messages = lambda x :[
#    {"role": "system", "content": f"{template}"},
#    {"role": "user", "content": f"{x}"},
#]

def func_messages(text:str, template:str):
    WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))
    messages = lambda x: f"[INST] {template}{x}[/INST] \n"
    return messages(WHITESPACE_HANDLER(text))


In [41]:
response = pipeline("text-generation", model=model, max_new_tokens=1000, tokenizer=tokenizer)

In [122]:
text = func_messages(data[0].page_content, template)

In [106]:
inputs = tokenizer(text, return_tensors="pt").to(device)

In [97]:
len(inputs["input_ids"][0])

945

In [39]:
test = response(messages(data[0].page_content))
test

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': '<s>[INST] Eres un asistente de facturas, solo hablas español, vas a recibir\n    información de varias facturas y debes extraer la siguiente información y presentarla\n    de esta forma y solo con estos datos:\n\n    Empresa: Nombre de la empresa del documento\n    Nit: Nit de la empresa del documento\n    Valor a pagar: Valor a pagar mostrado en el documento\n    Numero de factura: Numero de factura encontrado en el documento\n    Valor a pagar: Valor a pagar\n    IVA: Valor de iva\n    CUFE: CUFE del documento\n\n    A partir del siguiente documento:\n\n     FACTURA ELECTRÓNICA DE VENTA\n\nSODIMAC COLOMBIA S.A. NIT 800.242.106-2 Iva Régimen Común ICA Actividad Económica CIIU 4719 Grandes Contribuyentes Resolución 12220 26 DIC 2022 AUTORRETENEDORES Resolución DIAN 00931 de 29 ENE 2009 Agente de Retención de IVA\n\nN° 4008100243777\n\nINFORMACIÓN DEL CLIENTE\n\nRazón Social:\n\nNIT\n\nDirección\n\n: Armando Lopez Florian : 71.624.785 : CL 24 79 A 140\n\nFecha de Ex

In [113]:
with torch.inference_mode():
    outputs = model.generate(input_ids=inputs["input_ids"]
    , attention_mask=inputs["attention_mask"]
    , max_new_tokens=200
    ,do_sample=True
    ,top_p=0.95
    ,top_k=40
    ,repetition_penalty=1.1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [114]:
respuesta = "".join(tokenizer.batch_decode(outputs[0]))

In [54]:
from transformers import AutoModelForSeq2SeqLM

In [56]:
model_name = "csebuetnlp/mT5_multilingual_XLSum"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name,
                                             device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="main") # which version of mode

ReadTimeout: (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: f8487a04-c751-45b4-ba4d-959fe99d13b1)')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [50]:
response = pipeline("summarization", model=model, max_new_tokens=500, tokenizer=tokenizer)

The model 'BartForCausalLM' is not supported for summarization. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


In [ ]:
import re

In [43]:
template="""Eres un asistente de facturas, solo hablas español, vas a recibir
información de varias facturas y debes extraer la siguiente información y presentarla

de esta forma y solo con estos datos:

Empresa: Nombre de la empresa del documento
Nit: Nit de la empresa del documento
Valor a pagar: Valor a pagar mostrado en el documento
Numero de factura: Numero de factura encontrado en el documento
Valor a pagar: Valor a pagar
Valor de IVA: Valor de iva
CUFE: CUFE del documento

A partir del siguiente documento:

"""
#messages = lambda x :[
#    {"role": "system", "content": f"{template}"},
#    {"role": "user", "content": f"{x}"},
#]
messages = lambda x: f"{template} {x}"

In [51]:
with torch.inference_mode():
    outputs = response(messages(data[0].page_content))

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (1024). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.
